In [1]:
from google.cloud import bigquery
import time
bq = bigquery.Client()

In [2]:
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [3]:
#q1
q1 = bq.query(
    """
    SELECT geo_id
    FROM bigquery-public-data.geo_us_boundaries.counties
    WHERE county_name = 'Dane'
    """,
    job_config=bigquery.QueryJobConfig(use_query_cache=False)
)
dane_geo_id = q1.to_dataframe().loc[0, 'geo_id']
dane_geo_id

'55025'

In [4]:
#q2
q2 = bq.query(
    """
    SELECT state_fips_code, COUNT(*) as num_counties
    FROM bigquery-public-data.geo_us_boundaries.counties
    GROUP BY state_fips_code
    ORDER BY num_counties DESC
    LIMIT 5
    """,
    job_config=bigquery.QueryJobConfig(use_query_cache=False)
)
counties_per_state = q2.to_dataframe().set_index('state_fips_code')['num_counties'].to_dict()
counties_per_state

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [5]:
#q3
cost_per_tb_iowa = 0.02  # Replace with the actual cost per terabyte for Iowa
bytes_billed_q1 = q1.total_bytes_billed
bytes_billed_q2 = q2.total_bytes_billed

cost_q1 = (bytes_billed_q1 / (1024 ** 4)) * cost_per_tb_iowa
cost_q2 = (bytes_billed_q2 / (1024 ** 4)) * cost_per_tb_iowa

answer = {'q1': cost_q1, 'q2': cost_q2}
answer

{'q1': 1.9073486328125e-07, 'q2': 1.9073486328125e-07}

In [6]:
project = 'cs-544-398216'
dataset = 'p8'
table = 'hdma'
ds = bigquery.Dataset(f'{project}.{dataset}')
bq.create_dataset(ds,exists_ok=True)
table_id = f'{project}.{dataset}.{table}'
url = 'gs://cs544_p8_ana/hdma-wi-2021.parquet'

# Define the job configuration
config = bigquery.LoadJobConfig(
    source_format="PARQUET",
    write_disposition="WRITE_TRUNCATE"
)

# Load the Parquet data into a BigQuery table
job = bq.load_table_from_uri(url, table_id, job_config=config)
job.result()

LoadJob<project=cs-544-398216, location=US, id=91735cca-fe09-4cce-98ae-78fd180451ad>

In [7]:
#q4
[ds.dataset_id for ds in bq.list_datasets(project)] # PASTE your project name

['p8']

In [8]:
#q5
query_top_counties = """
WITH TopCounties AS (
  SELECT
    counties.county_name,
    COUNT(*) AS loan_count
  FROM
    cs-544-398216.p8.hdma AS hdma
  JOIN
    bigquery-public-data.geo_us_boundaries.counties AS counties
  ON
    counties.county_fips_code = hdma.county_code
  GROUP BY
    counties.county_name
  ORDER BY
    loan_count DESC
  LIMIT 10
)

SELECT
  county_name,
  loan_count
FROM
  TopCounties
"""

result_top_counties = bq.query(query_top_counties)

top_counties_dict = {row['county_name']: row['loan_count'] for row in result_top_counties}
top_counties_dict

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [9]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(f'{project}.{dataset}.applications')
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [10]:
#q6
query_string = """
SELECT
  count(*) as count
FROM
  cs-544-398216.p8.applications
WHERE income = 80000
"""
bq.query(query_string).to_dataframe().iloc[0]['count']

2

In [11]:
#q7
query = """
WITH Points AS (
  SELECT
    ST_GEOGPOINT(longitude, latitude) AS point,
    county_name
  FROM
    cs-544-398216.p8.applications AS sheet
  JOIN
    bigquery-public-data.geo_us_boundaries.counties AS counties
  ON
    ST_CONTAINS(counties.county_geom, ST_GEOGPOINT(longitude, latitude))
    AND counties.state_fips_code = '55'
)

SELECT
  county_name,
  COUNT(*) AS count
FROM
  Points
GROUP BY
  county_name
"""

# Execute the query
query_job = bq.query(query)

{row['county_name']: row['count'] for row in query_job}

{'Door': 4,
 'Walworth': 2,
 'Sheboygan': 2,
 'Bayfield': 1,
 'Kewaunee': 1,
 'Monroe': 1,
 'Barron': 2,
 'Outagamie': 1,
 'Columbia': 1,
 'Brown': 2,
 'Oneida': 1,
 'Dane': 8,
 'Green Lake': 1,
 'Marinette': 2,
 'Sauk': 1,
 'Jefferson': 1}

In [12]:
model = 'loan_model'
table = 'hdma'
linear_regression_query = f"""
CREATE OR REPLACE MODEL `{project}.{dataset}.{model}`
OPTIONS(model_type='linear_reg') AS
SELECT
  income,
  loan_term,
  loan_amount
FROM
  `{project}.{dataset}.{table}`
"""

# Execute the linear regression query
query_job = bq.query(linear_regression_query)

In [13]:
# Wait until the model is created (polling)
count = 0
while True:
    models = list(bq.list_models(f'{project}.{dataset}'))
    if any(model_n.model_id == model for model_n in models):
        break
    time.sleep(5)
    count += 5
    if count > 120:
        break

In [14]:
#q8
evaluation_query = f"""
SELECT
  loan_amount AS true_loan_amount,
  predicted_loan_amount
FROM
  ML.PREDICT(MODEL `{project}.{dataset}.{model}`,
    (
      SELECT
        income,
        loan_term,
        loan_amount
      FROM
        `{project}.{dataset}.{table}`
    )
  )
"""

# Execute the evaluation query
predictions = bq.query(evaluation_query).to_dataframe()

# Calculate r2_score
r2 = r2_score(predictions['true_loan_amount'], predictions['predicted_loan_amount'])
r2

NotFound: 404 Not found: Model cs-544-398216:p8.loan_model

Location: US
Job ID: a9a6c27f-27c3-4591-bdf3-39a62f0d85c6


In [ ]:
#q9
weights_query = f"""
SELECT
  *
FROM
  ML.WEIGHTS(MODEL `{project}.{dataset}.{model}`)
"""

# Execute the query
weights_result = bq.query(weights_query).to_dataframe()

# Extract the coefficient weight on the 'income' column
weights_result.loc[weights_result['input'] == 'income', 'weight'].values[0]

In [ ]:
#q10
# Query to get predictions from the model for Google Form data
google = 'applications'
predictions_query = f"""
SELECT
  income,
  360 AS loan_term,  -- Assume loan term is 360 months (30 years)
  ML.PREDICT(MODEL `{project}.{dataset}.{model}`,
    STRUCT(income, 360) AS features) AS predicted_loan_amount
FROM
  `{project}.{dataset}.{google}`
"""

# Execute the query
predictions_df = bq.query(predictions_query).to_dataframe()

# Calculate the ratio of loan applications with amounts greater than predicted
total_applications = len(predictions_df)
greater_than_predicted = len(predictions_df[predictions_df['actual_loan_amount'] > predictions_df['predicted_loan_amount']])

ratio_greater_than_predicted = greater_than_predicted / total_applications